In [ ]:
# import quantum circuit for Deutsch-Jozsa algorithm
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2  as Sampler
# Initialize the Qiskit Runtime Service
from test_utils import read_account_file

In [ ]:
account = read_account_file('account_info.conf')
service = QiskitRuntimeService(channel="ibm_cloud", instance=account['instance'], token=account['token'])
backend = service.least_busy()
print(f"Using backend: {backend.name}")

In [ ]:
# Create a quantum circuit for Deutsch-Jozsa algorithm
qc = QuantumCircuit(5, 1)  # 5 qubits, 1 classical bit
qc.h(range(4))  # Apply Hadamard gate to the first 4 qubits
qc.x(4)  # Set the last qubit to |1>
qc.h(4)  # Apply Hadamard gate to the last qubit
qc.barrier()  # Add a barrier for clarity
qc.draw("mpl")  # Draw the circuit
#Create a random oracle for Deutsch-Jozsa algorithm
def random_oracle(case,qc):
    if case == "constant":
        # Implement a constant oracle f(x) = 0
        pass  # No gates needed for constant function
    elif case == "balanced":
        # Implement a balanced oracle f(x) = x1 XOR x2
        qc.cx(0, 4)
        qc.cx(1, 4)
        qc.cx(2, 4)
        qc.cx(3, 4)
    else:
        raise ValueError("Invalid case. Use 'constant' or 'balanced'.")

# Apply the random oracle to the circuit
#random_oracle("constant", qc)  # Use "constant" or "balanced" as needed
random_oracle("balanced", qc)  # Use "constant" or "balanced" as needed
qc.barrier()  # Add a barrier after the oracle
# Apply Hadamard gates again to the first 4 qubits
qc.h(range(4))
# Measure the first qubit
qc.measure(0, 0)  # Measure the first qubit into the classical bit
qc.draw("mpl")  # Draw the circuit after measurement

In [ ]:
# Transpile the circuit using a preset pass manager
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
# Transpile the circuit
transpiled_circuit = pm.run(qc)
transpiled_circuit.draw("mpl")  # Draw the transpiled circuit
# Create a sampler instance
sampler = Sampler(mode=backend)
# Sample the transpiled circuit
job = sampler.run([transpiled_circuit], shots=1024)
print(f"Job ID: {job.job_id()}")

In [ ]:
# Get the results
results = job.result()
# Plot the histogram of results using cregisters
counts = results[0].data.c.get_counts()
# Plot the histogram
plot_histogram(counts, title="Deutsch-Jozsa Algorithm Results")

